# Multi Unit Activity Analysis


### 1) Load Required Packages 

In [1]:
#Import required packages
import pickle
import os
import ipywidgets
import csv
import numpy as np
from ipywidgets import Layout, HBox, VBox
from IPython.display import display
from load_probe_info import *
from cycleDirectoryMUA_Simon import *

### 2) Provide Information for Dictionary 

In [2]:
##Main path for the data 

inputPath_html = ipywidgets.HTML(value = "<p><b>Path to the data of the experiment:</b><br />Enter the path to the folder (with no '/' at the end) that is hierarchically right above the folders of the recording sessions</p>")
inputPath = ipywidgets.Text(value = "/mnt/d/DatainAnalysis", placeholder = "Enter path for data", disabled = False)
display(VBox([inputPath_html, inputPath]))

##Main path for the output results and figures 

outputPath_html = ipywidgets.HTML(value = "<p><b>Path for the resulting analysis and figures:</b><br />Enter the path to the folder (with '/') where all results should be stored </p>")
outputPath = ipywidgets.Text(value = "/mnt/c/Users/gwendolyn/Documents/ActiveAnalysis/AnalysisOutput", placeholder = "Enter path for data", disabled = False)
display(VBox([outputPath_html, outputPath]))

##Sampling rate
sr = ipywidgets.IntText(value = 32000, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b> Sampling rate (Hz): </b>"),sr]))

##Probe info
pi_html = ipywidgets.HTML(value = "<b>Type of the probe used in the experiment</b>")
pi = ipywidgets.Dropdown(options=['a2x16_10mm_100_500_177', 'a2x16_10mm_50_500_177', 'a1x32_6mm_100_177', 'a4x8_5mm_200_400_177'], 
                   value = 'a2x16_10mm_100_500_177',  disabled = False)
display(VBox([pi_html, pi]))

##TimeWindow

tw = ipywidgets.Dropdown(options = [('-0.050-0.20', 1), ('-0.010-0.050', 2)], value = 1, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Select the time window for analysis(in s)</b>"), tw]))

#High_pass_freq

hp = ipywidgets.FloatText(value = 800, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b> Enter the cutoff frequency of the high pass filter to extract MUA from data (in Hz)"), hp]))

#Thresold

th = ipywidgets.FloatText(value = 8, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b> Enter the multiple of the standard deviation used for thresholding the data)"), th]))


#PSTH binsize 
psth_bs = ipywidgets.Dropdown(options = [('5', 1), ('10',2)], value = 1,  disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Select the time window for PSTH analysis (in ms)</b>"), psth_bs]))

VBox(children=(HTML(value="<p><b>Path to the data of the experiment:</b><br />Enter the path to the folder (with no '/' at the end) that is hierarchically right above the folders of the recording sessions</p>"), Text(value='/mnt/d/DatainAnalysis', placeholder='Enter path for data')))

VBox(children=(HTML(value="<p><b>Path for the resulting analysis and figures:</b><br />Enter the path to the folder (with '/') where all results should be stored </p>"), Text(value='/mnt/c/Users/gwendolyn/Documents/ActiveAnalysis/AnalysisOutput', placeholder='Enter path for data')))

VBox(children=(HTML(value='<b> Sampling rate (Hz): </b>'), IntText(value=32000)))

VBox(children=(HTML(value='<b>Type of the probe used in the experiment</b>'), Dropdown(options=('a2x16_10mm_100_500_177', 'a2x16_10mm_50_500_177', 'a1x32_6mm_100_177', 'a4x8_5mm_200_400_177'), value='a2x16_10mm_100_500_177')))

VBox(children=(HTML(value='<b>Select the time window for analysis(in s)</b>'), Dropdown(options=(('-0.050-0.20', 1), ('-0.010-0.050', 2)), value=1)))

VBox(children=(HTML(value='<b> Enter the cutoff frequency of the high pass filter to extract MUA from data (in Hz)'), FloatText(value=800.0)))

VBox(children=(HTML(value='<b> Enter the multiple of the standard deviation used for thresholding the data)'), FloatText(value=8.0)))

VBox(children=(HTML(value='<b>Select the time window for PSTH analysis (in ms)</b>'), Dropdown(options=(('5', 1), ('10', 2)), value=1)))

### 3) Write Dictionary 

In [4]:
p = {} #Parameter dictionary (empty)

#Entering the probe info and electrode mapping into the dictionary
probe_info = load_probe_info(pi.value)
p['shanks'] = probe_info['numShanks']

p['probe_name'] = probe_info['name']
p['nr_of_electrodes'] = probe_info['numTrodes']
p['nr_of_electrodes_per_shank'] = probe_info['numTrodesPerShank']
p['nr_of_shanks'] = p['shanks']
p['bottom_ycoord'] = probe_info['bottom_ycoord']
p['top_ycoord'] = probe_info['top_ycoord']
p['id'] = probe_info['id']
p['sitespacing'] = probe_info['sitespacing']

#Entering the path and file format info into the dictionary
p['inputPath'] = inputPath.value
p['outputPath'] = outputPath.value

#Entering the general parameters into the dictionary
p['sample_rate'] = sr.value
    
#Entering the MUA analysis parameters into the dictionary
if tw.value == 1:
    p['evoked_pre'] = 0.05
    p['evoked_post'] = 0.20
if tw.value == 2:
    p['evoked_pre'] = 0.01
    p['evoked_post'] = 0.05
p['high_pass_freq'] = hp.value
p['threshold'] = th.value 
if psth_bs.value == 1:
    p['psth_binsize'] = 5 
if psth_bs.value == 2:
    p['psth_binsize'] = 10 
        
if not os.path.exists(outputPath.value + '/AnalysisFiles'):
    os.mkdir(outputPath.value + '/AnalysisFiles')
    
#Saving the dictionary in the pickle file and csv named parametersDict
pickle.dump(p, open((outputPath.value + '/AnalysisFiles/parametersDict.p'), 'wb'))

with open(outputPath.value + '/AnalysisFiles/parametersDict.csv', 'w') as textfile:
    fieldnames = ['Field', 'Value']
    writer = csv.DictWriter(textfile, fieldnames = fieldnames)
    writer.writeheader()
    data = [dict(zip(fieldnames, [k,v])) for k, v in p.items()]
    writer.writerows(data)
    
print(p)


loadding
{'shanks': 1, 'probe_name': 'a1x32_6mm_100_177', 'nr_of_electrodes': 32, 'nr_of_electrodes_per_shank': 32, 'nr_of_shanks': 1, 'bottom_ycoord': 0, 'top_ycoord': 3200, 'id': array([[ 1, 32,  3, 30,  5, 28,  7, 26,  2, 31,  6, 17,  4, 29,  9, 18,
         8, 27, 11, 19, 10, 25, 13, 20, 12, 24, 15, 21, 14, 23, 16, 22]]), 'sitespacing': 100, 'inputPath': '/mnt/e/UnpackedDatainAnalysis/mGE82838485', 'outputPath': '/mnt/d/ActiveAnalysis/AnalysisOutput/', 'sample_rate': 32000, 'evoked_pre': 0.05, 'evoked_post': 0.2, 'high_pass_freq': 800.0, 'threshold': 8.0, 'psth_binsize': 5}


### 4) Complete Analysis

In [ ]:
MUA_analyzeAllFiles(p)

mGE85_31.07.2019_O10C1.mcd
Triggers_Deviant.dat


/home/gwendolyn/anaconda3/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:2386: RuntimeWarning: Mean of empty slice.
  xbar = data.mean()
/home/gwendolyn/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/gwendolyn/anaconda3/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:2403: RuntimeWarning: Mean of empty slice.
  s = np.log(xbar) - np.log(data).mean()
/home/gwendolyn/anaconda3/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:187: RuntimeWarning: Mean of empty slice.
  loc = data.mean()
/home/gwendolyn/anaconda3/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:192: RuntimeWarning: Mean of empty slice.
  scale = np.sqrt(((data - loc)**2).mean())
/mnt/c/Users/gwendolyn/Documents/EphysPipeline/gwendolyn_ephys_updated/mainAnalysisMUA_Simon.py:81: RuntimeWarning: Mean of empty slice.
  avgTimetoFirstSpike = mins[np.nonze

Triggers_Standard.dat
Triggers_Predeviant.dat
Triggers_Postdeviant.dat
mGE85_31.07.2019_O10C2.mcd
Triggers_Deviant.dat
Triggers_Standard.dat
Triggers_Predeviant.dat
